In [48]:
import numpy as np
import matplotlib.pyplot as plt

In [130]:
class State():
    def __init__(self, money=0, start_time=0):
        self.money = money
        self.time = start_time

    
class Action():
    def __init__(self, message, state_change, child, prob=None):
        self.message = message
        self.state_change = state_change
        self.child = child
        self.prob = prob
    
    def apply_state_change(self, state):
        return self.state_change(state)

    def __repr__(self):
        return f'action: {self.message}'


class Node():
    def __init__(self, message, actions):
        self.message = message
        self.actions = actions

class RandomNode(Node):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.type = 'random'
    
    def do_action(self, state):
        probs = [a.prob for a in self.actions]
        action_id = np.random.choice(range(len(self.actions)), p=probs)
        action = self.actions[action_id]
        new_state = action.apply_state_change(state)
        return new_state, action.child, action.message

class ActionNode(Node):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.type = 'action'

    @property
    def action_messages(self):
        for i, action in enumerate(self.actions):
            yield (i, action.message)

    def do_action(self, state):
        for i, msg in self.action_messages:
            print(f'{i}: ', end='')
            if msg:
                print(msg)
        act_id = int(input('choose id'))
        action = self.actions[act_id]
        new_state = action.apply_state_change(state)
        return new_state, action.child, None

In [131]:
def state_change_generator(money, time):
    def state_change(state):
        state.money += money
        state.time += time
        return state
    return state_change

In [132]:
class World():
    def __init__(self, nodes):
        self.nodes = nodes
        self.current_node = self.nodes[0]
        self.current_state = State(1000, 0)

    def time_step(self):
        if self.current_node.message:
            print(self.current_node.message)

        self.current_state, new_node_id, action_message = self.current_node.do_action(self.current_state)
        if action_message:
            print(action_message)
        
        self.current_node = self.nodes[new_node_id]
    
    def __repr__(self, ):
        return f'current state: {self.current_state.money} ТФ Рублей. Сейчас {self.current_state.time} игровой день'

In [133]:
def node_parser(raw_node):
    actions = []
    for raw_action in raw_node['actions']:
        if raw_node['type'] == 'action':
            prob = None
        else:
            prob = raw_action['prob']
        actions.append(
            Action(
                raw_action['message'],
                state_change_generator(
                    raw_action['state_change']['money'],
                    raw_action['state_change']['time']
                ),
                raw_action['child'],
                prob
            )
        )

    if raw_node['type'] == 'action':
        return ActionNode(raw_node['message'], actions)
    else:
         return RandomNode(raw_node['message'], actions)



In [139]:
import yaml

with open('action_tree.yaml', 'r') as f:
    raw_nodes = yaml.load(f.read())['nodes']

nodes = [node_parser(n) for n in raw_nodes]

In [140]:
world = World(nodes)
world.time_step()
world.time_step()
world.time_step()
world.time_step()
world.time_step()
world.time_step()
world.time_step()
world.time_step()

Ну что, пойдем на работу?
0: Да
1: Нет
Ты упал с лестницы, теперь нужны деньги на лекарства. 500 ТФ рублей не мало, но на здоровье экономить нельзя
Ну что, пойдем на работу?
0: Да
1: Нет
Ты упал с лестницы, теперь нужны деньги на лекарства. 500 ТФ рублей не мало, но на здоровье экономить нельзя
Ну что, пойдем на работу?
0: Да
1: Нет
Ты получил наследство от далекого дяди, которого ты никогда не знал. Ужасная утрата, надеюсь эти 1000 ТФ рублей смогут помочь.
Ну что, пойдем на работу?
0: Да
1: Нет
Ты получил наследство от далекого дяди, которого ты никогда не знал. Ужасная утрата, надеюсь эти 1000 ТФ рублей смогут помочь.


In [143]:
world

current state: 2400 ТФ Рублей. Сейчас 8 игровой день